## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dockets/DOCKET_FDA-2019-N-1482.csv', header=2)

In [3]:
df.columns

Index(['Document Title', 'Document Type', 'Attachment Count',
       'Submitter Last Name', 'Submitter First Name', 'Organization',
       'Document ID', 'Posted Date', 'Received Date', 'Comment Start Date',
       'Comment Due Date', 'Implementation Date', 'Effective Date',
       'Related RINs', 'Document SubType', 'Subject', 'Abstract', 'Status',
       'Source Citation', 'OMB Approval Number', 'FR Citation',
       'Federal Register Number', 'Start End Page', 'Special Instructions',
       'Legacy ID', 'Post Mark Date', 'File Type', 'Number Of Pages',
       'Paper Width', 'Paper Length', 'Exhibit Type', 'Exhibit Location',
       'Document Field_1', 'Document Field_2', 'Document Detail'],
      dtype='object')

In [4]:
df.head()

,Document Title,Document Type,Attachment Count,Submitter Last Name,Submitter First Name,Organization,Document ID,Posted Date,Received Date,Comment Start Date,...,Post Mark Date,File Type,Number Of Pages,Paper Width,Paper Length,Exhibit Type,Exhibit Location,Document Field_1,Document Field_2,Document Detail
0,Comment from Ben Reusser,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,FDA-2019-N-1482-0662,05/21/2019,05/18/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=FDA-201...
1,Comment from Ron Littleton,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,FDA-2019-N-1482-0220,04/10/2019,04/07/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=FDA-201...
2,Comment from Anonymous,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,FDA-2019-N-1482-0668,05/21/2019,05/18/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=FDA-201...
3,Comment from Lance Schroeder,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,FDA-2019-N-1482-1851,06/21/2019,06/18/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=FDA-201...
4,Comment from Robert Keller,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,FDA-2019-N-1482-0344,04/11/2019,04/11/2019,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=FDA-201...


In [5]:
import requests
import webbrowser 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import re

In [6]:
trial = df['Document Detail'][:3]
trial

0    https://www.regulations.gov/document?D=FDA-201...
1    https://www.regulations.gov/document?D=FDA-201...
2    https://www.regulations.gov/document?D=FDA-201...
Name: Document Detail, dtype: object

In [7]:
trial[0]

'https://www.regulations.gov/document?D=FDA-2019-N-1482-0662'

In [8]:
trial[1]

'https://www.regulations.gov/document?D=FDA-2019-N-1482-0220'

https://scrapoxy.io/ For the rotating proxies for Scrapy

Too complicated to work

In [ ]:
from lxml.html import fromstring
import requests
from itertools import cycle
import traceback

def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):                     #HTTP check
            #if i.xpath('.//td[5][contains(text(),"elite proxy")]'):         #Elite check
            
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])  #Grabbing IP and corresponding PORT
            proxies.add(proxy)
    return proxies

/html/body/section[1]/div/div[2]/div/div[2]/div/table/tbody/tr[1]/td[5]

In [ ]:
proxies = get_proxies()
proxy_pool = cycle(proxies)

In [ ]:
proxies

In [ ]:
url = 'https://httpbin.org/ip'
good_proxy = []
for i in range(1,11):
    #Get a proxy from the pool
    proxy = next(proxy_pool)
    print("Request #%d"%i)
    try:
        response = requests.get(url,proxies={"http": proxy, "https": proxy})
        print(response.json())
        good_proxy.append(proxy)
    except:
        #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
        #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
        print("Skipping. Connnection error")

In [ ]:
good_proxy

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType

proxies = list(proxies) #Turns set into list


prox = Proxy()
prox.proxy_type = ProxyType.MANUAL
prox.http_proxy = proxies[0]
prox.socks_proxy = proxies[0]
prox.ssl_proxy = proxies[0]

capabilities = webdriver.DesiredCapabilities.CHROME
prox.add_to_capabilities(capabilities)

driver = webdriver.Chrome(desired_capabilities=capabilities)

In [ ]:
import os

chromedriver = '/home/socjon/DS/FTC-Comments/chromedriver'   #Had to specify the entire PATH
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.implicitly_wait(30)
comments = []
for url in df['Document Detail']:
    driver.get(url)
    time.sleep(1) #gives time for the page to load.
    soup_level1 = BeautifulSoup(driver.page_source)
    huh = soup_level1.find('div', class_="GIY1LSJIXD")
    comments.append(huh)

In [ ]:
len(comments)

In [ ]:


chrome_options = Options()
chrome_options.add_argument('--dns-prefetch-disable')
driver = Chrome(chrome_options=chrome_options)

Timed out. Since I was using one session, I have to either restate

In [13]:

huh += 1
huh

2

In [13]:
import os
from selenium.webdriver.chrome.options import Options

counter = 0
comments = []


chromedriver = '/home/socjon/DS/FTC-Comments/chromedriver'   #Had to specify the entire PATH
os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = Options()
chrome_options.add_argument('--dns-prefetch-disable')
driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)
driver.implicitly_wait(30)



for url in df['Document Detail']:
    if (counter % 100) == 0: 
        driver.quit()
        
        chromedriver = '/home/socjon/DS/FTC-Comments/chromedriver'   #Had to specify the entire PATH
        os.environ["webdriver.chrome.driver"] = chromedriver
        chrome_options = Options()
        chrome_options.add_argument('--dns-prefetch-disable')
        driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)
        driver.implicitly_wait(30)

    
    driver.get(url)
    time.sleep(1) #gives time for the page to load.
    soup_level1 = BeautifulSoup(driver.page_source)
    huh = soup_level1.find('div', class_="GIY1LSJIXD")
    comments.append(huh)
    counter += 1

/home/socjon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':
/home/socjon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


In [14]:
len(comments)

2070

In [19]:
import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(comments, fp)

RecursionError: maximum recursion depth exceeded

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType

try:
    for url in df['Document Detail']:

        for proxy in good_proxy:

            prox = Proxy()
            prox.proxy_type = ProxyType.MANUAL
            prox.http_proxy = proxy
            prox.socks_proxy = proxy
            prox.ssl_proxy = proxy

            capabilities = webdriver.DesiredCapabilities.CHROME
            prox.add_to_capabilities(capabilities)

            driver = webdriver.Chrome(desired_capabilities=capabilities)
            driver.implicitly_wait(30)
            comments = []

            driver.get(url)
            time.sleep(1) #gives time for the page to load.
            soup_level1 = BeautifulSoup(driver.page_source)
            huh = soup_level1.find('div', class_="GIY1LSJIXD")
            comments.append(huh)
            break
except:
    print("Huh")

In [ ]:

for i in list(range(1,11)):
    for g in list(range(1,11)):
        try:
            
        else:
            pass

In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(30)
comments = []

    driver.get(url)
    time.sleep(1) #gives time for the page to load.
    soup_level1 = BeautifulSoup(driver.page_source)
    huh = soup_level1.find('div', class_="GIY1LSJIXD")
    comments.append(huh)

In [ ]:
len(comments)

## Cleaning

In [ ]:
pattern = re.compile(r"(?<=\"\"\>).*?(?=\<\/div>)")

In [ ]:
pattern.findall(str(comments[1]))

**BAM!**

In [ ]:
wait_time = 1/60
num_comments = 100000 #100K

time_it_will_take_in_hours = wait_time*num_comments

In [ ]:
soup_level1=BeautifulSoup(driver.page_source)

In [ ]:
huh = soup_level1.find('div', class_="GIY1LSJIXD")
huh

In [ ]:
post_feed = source.find('div', class_="")

In [ ]:
url = requests.get(df['Document Detail'][1])
browser  = webdriver.Chrome(executable_path='chromedriver.exe')

browser.implicitly_wait(30)
browser.get(url)
